In [5]:
import cv2
import numpy as np

def count_and_display_black_squares(image_path):
    # Lade das Bild
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        raise FileNotFoundError(f"Bild konnte nicht geladen werden: {image_path}")
    
    # Thresholding, um das Schachbrettmuster zu segmentieren
    _, binary = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Finde horizontale und vertikale Linien
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    
    horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel)
    
    # Kombiniere die Linien, um das Gitter zu finden
    grid = cv2.add(horizontal_lines, vertical_lines)
    
    # Finde die Schnittpunkte des Gitters
    intersections = cv2.bitwise_and(horizontal_lines, vertical_lines)
    
    # Finde die Positionen der Schnittpunkte
    contours, _ = cv2.findContours(intersections, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    points = [cv2.boundingRect(cnt)[:2] for cnt in contours]
    
    if len(points) < 64:
        raise ValueError(f"Nicht genügend Schnittpunkte erkannt: {len(points)} gefunden, 64 erwartet.")
    
    # Sortiere die Punkte und filtere sie auf 64 Punkte (8x8)
    points = np.array(points, dtype="float32")
    points = points[np.argsort(points[:, 1])]  # Nach y-Koordinate sortieren
    grid_points = []
    for i in range(8):  # In 8 Zeilen aufteilen
        row = points[i * (len(points) // 8):(i + 1) * (len(points) // 8)]
        row = row[np.argsort(row[:, 0])]  # Nach x-Koordinate sortieren
        grid_points.append(row)
    grid_points = np.array(grid_points)
    
    if grid_points.shape != (8, 8, 2):
        raise ValueError(f"Unerwartete Form der Gitterpunkte: {grid_points.shape}")
    
    # Perspektivkorrektur
    top_left = grid_points[0, 0]
    top_right = grid_points[0, -1]
    bottom_left = grid_points[-1, 0]
    bottom_right = grid_points[-1, -1]
    
    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    side = int(max(np.linalg.norm(top_right - top_left), np.linalg.norm(bottom_right - bottom_left)))
    dst_pts = np.array([[0, 0], [side, 0], [side, side], [0, side]], dtype="float32")
    
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, matrix, (side, side))
    
    # Teile das Schachbrett in Zellen auf
    cell_size = warped.shape[0] // 8
    black_square_count = 0
    
    # Kopie des Bildes für die Visualisierung
    warped_colored = cv2.cvtColor(warped, cv2.COLOR_GRAY2BGR)
    
    for row in range(8):
        for col in range(8):
            # Extrahiere jede Zelle
            cell = warped[row * cell_size:(row + 1) * cell_size, col * cell_size:(col + 1) * cell_size]
            # Berechne den Durchschnittswert der Zelle
            mean_intensity = np.mean(cell)
            # Wenn der Durchschnittswert niedrig ist, ist es ein schwarzes Quadrat
            if mean_intensity < 128:
                black_square_count += 1
                # Zeichne ein Rechteck um das schwarze Quadrat
                top_left = (col * cell_size, row * cell_size)
                bottom_right = ((col + 1) * cell_size, (row + 1) * cell_size)
                cv2.rectangle(warped_colored, top_left, bottom_right, (0, 0, 255), 2)
    
    # Zeige das Bild mit hervorgehobenen schwarzen Quadraten
    cv2.imshow("Erkannte schwarze Quadrate", warped_colored)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return black_square_count

# Beispielaufruf
image_path = 'image0.jpg'
try:
    black_squares = count_and_display_black_squares(image_path)
    print(f"Anzahl der schwarzen Quadrate: {black_squares}")
except Exception as e:
    print(f"Fehler: {e}")

Fehler: Unerwartete Form der Gitterpunkte: (8, 9, 2)
